In [ ]:
# this section is just gettign all of the liabarys that are needed 
library(tidyverse)
library(rvest)
library(polite) # this library is jsut a better version of the rvest library 
library(xml2) # makes it easier to work with HTML and XML from R
library(glue)
library(purrr)